In [1]:
import requests
from bs4 import BeautifulSoup
from nba_api.stats.endpoints import leaguedashteamstats, leaguedashteamshotlocations, commonallplayers
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [25]:
# Create an instance of the CommonAllPlayers endpoint
all_players_instance = commonallplayers.CommonAllPlayers()

# Retrieve the data
all_players = all_players_instance.get_data_frames()[0]  # Get the first DataFrame

# Extract relevant data
all_players = all_players[all_players['ROSTERSTATUS'] != 0]
all_players = all_players[['DISPLAY_FIRST_LAST', 'TEAM_CITY', 'TEAM_NAME']]
all_players['FIRST_NAME'] = all_players['DISPLAY_FIRST_LAST'].str.split().str[0]
all_players['LAST_NAME'] = all_players['DISPLAY_FIRST_LAST'].str.split().str[1]
all_players = all_players.drop(columns=['DISPLAY_FIRST_LAST'], axis =1)
all_players.to_csv('players.csv', index=False)

In [6]:
# Create an instance of the LeagueDashTeamStats endpoint
team_shot_locations_connect = leaguedashteamshotlocations.LeagueDashTeamShotLocations()

# Retrieve the data
team_shot_locations_data = team_shot_locations_connect.get_data_frames()[0]  # Get the first DataFrame

team_shot_locations = pd.DataFrame(team_shot_locations_data)
team_shot_locations = team_shot_locations

teams = team_shot_locations['']['TEAM_NAME']

rim_data = team_shot_locations[['Restricted Area']]
rim_data = rim_data['Restricted Area'][['FGM', 'FGA', 'FG_PCT']]

short_mid_data = team_shot_locations[['In The Paint (Non-RA)']]
short_mid_data = short_mid_data['In The Paint (Non-RA)'][['FGM', 'FGA', 'FG_PCT']]

long_mid_data = team_shot_locations[['Mid-Range']]
long_mid_data = long_mid_data['Mid-Range'][['FGM', 'FGA', 'FG_PCT']]

corner_3_data = team_shot_locations[['Corner 3']]
corner_3_data = corner_3_data['Corner 3'][['FGM', 'FGA', 'FG_PCT']]

above_the_break_data = team_shot_locations[['Above the Break 3']]
above_the_break_data = above_the_break_data['Above the Break 3'][['FGM', 'FGA', 'FG_PCT']]

rim = pd.concat([teams, rim_data], axis=1).set_index('TEAM_NAME')
short_mid = pd.concat([teams, short_mid_data], axis=1).set_index('TEAM_NAME')
long_mid = pd.concat([teams, long_mid_data], axis=1).set_index('TEAM_NAME')
corner_3 = pd.concat([teams, corner_3_data], axis=1).set_index('TEAM_NAME')
above_the_break = pd.concat([teams, above_the_break_data], axis=1).set_index('TEAM_NAME')

rim_label = 'RIM_'
short_mid_label = 'SM_'
long_mid_label = 'LM_'
corner_3_label = 'C3_'
above_the_break_label = 'ATB_'

    
rim.rename(columns={'FGM':f'{rim_label}FGM', 'FGA':f'{rim_label}FGA', 'FG_PCT':f'{rim_label}FG_PCT'}, inplace = True)
short_mid.rename(columns={'FGM':f'{short_mid_label}FGM', 'FGA':f'{short_mid_label}FGA', 'FG_PCT':f'{short_mid_label}FG_PCT'}, inplace = True)
long_mid.rename(columns={'FGM':f'{long_mid_label}FGM', 'FGA':f'{long_mid_label}FGA', 'FG_PCT':f'{long_mid_label}FG_PCT'}, inplace = True)
corner_3.rename(columns={'FGM':f'{corner_3_label}FGM', 'FGA':f'{corner_3_label}FGA', 'FG_PCT':f'{corner_3_label}FG_PCT'}, inplace = True)
above_the_break.rename(columns={'FGM':f'{above_the_break_label}FGM', 'FGA':f'{above_the_break_label}FGA', 'FG_PCT':f'{above_the_break_label}FG_PCT'}, inplace = True)

In [7]:
# Create an instance of the LeagueDashTeamStats endpoint
team_stats = leaguedashteamstats.LeagueDashTeamStats()

# Retrieve the data
team_stats_data = team_stats.get_data_frames()[0]  # Get the first DataFrame

# Grab the PPP necessary stats
ovr_PPP_data = team_stats_data[['TEAM_NAME', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'TOV']]
PPP_stats = ovr_PPP_data.copy()

# Calculate points for each team
PPP_stats['FG2_POINTS'] = (PPP_stats['FGM'] - PPP_stats['FG3M']) * 2
PPP_stats['FG3_POINTS'] = PPP_stats['FG3M'] * 3
PPP_stats['FT_POINTS'] = PPP_stats['FTM']

# Calculate total points scored by each team
PPP_stats['TOTAL_POINTS'] = PPP_stats['FG2_POINTS'] + PPP_stats['FG3_POINTS'] + PPP_stats['FT_POINTS']
PPP_stats = PPP_stats.set_index('TEAM_NAME')

League_Totals = PPP_stats.copy()
League_Totals = League_Totals.drop(['FG2_POINTS', 'FG3_POINTS', 'FT_POINTS'], axis=1)

# Calculate totals for all stats including points
totals = PPP_stats.select_dtypes(np.number).sum().rename('League Totals')
PPP_stats = pd.concat([PPP_stats, totals.to_frame().T])

# Calculate Points per Possession (PPP) for each team
total_poss = (PPP_stats['FGA'] + (0.44 * PPP_stats['FTA']) + PPP_stats['TOV'])
PPP_stats['PPP'] = round((PPP_stats['TOTAL_POINTS'] / (PPP_stats['FGA'] + (0.44 * PPP_stats['FTA']) + PPP_stats['TOV'])),2)
#PPP_stats[['PPP']].sort_values('PPP', ascending=False)

In [8]:
# Concat the Shot Zones to the league totals
League_Totals = pd.concat([League_Totals, rim, short_mid, long_mid, corner_3, above_the_break], axis=1)

# Calculate the league totals
totals = League_Totals.select_dtypes(np.number).sum().rename('League Totals')
League_Totals = pd.concat([League_Totals, totals.to_frame().T])

# Caculate the true FG_PCT for each shot zone
League_Totals['RIM_FG_PCT']['League Totals'] = League_Totals['RIM_FGM']['League Totals'] / League_Totals['RIM_FGA']['League Totals']
League_Totals['SM_FG_PCT']['League Totals'] = League_Totals['SM_FGM']['League Totals'] / League_Totals['SM_FGA']['League Totals']
League_Totals['LM_FG_PCT']['League Totals'] = League_Totals['LM_FGM']['League Totals'] / League_Totals['LM_FGA']['League Totals']
League_Totals['C3_FG_PCT']['League Totals'] = League_Totals['C3_FGM']['League Totals'] / League_Totals['C3_FGA']['League Totals']
League_Totals['ATB_FG_PCT']['League Totals'] = League_Totals['ATB_FGM']['League Totals'] / League_Totals['ATB_FGA']['League Totals']

# Calculate the frequency
League_Totals['RIM_FREQ'] = League_Totals['RIM_FGA'] / League_Totals['FGA']
League_Totals['SM_FREQ'] = League_Totals['SM_FGA'] / League_Totals['FGA']
League_Totals['LM_FREQ'] = League_Totals['LM_FGA'] / League_Totals['FGA']
League_Totals['C3_FREQ'] = League_Totals['C3_FGA'] / League_Totals['FGA']
League_Totals['ATB_FREQ'] = League_Totals['ATB_FGA'] / League_Totals['FGA']

In [10]:
League_Totals.to_csv('frequency.csv')

In [ ]:
League_Totals